In [1]:
ipython = get_ipython()
colab   = 'google.colab' in str(ipython)
print(ipython)

if colab:
    from google.colab import drive
    drive.mount('/content/gdrive')
    %cd '/content/gdrive/My Drive/Colab Notebooks'

!ls "./"

def MODULE_FROM_NOTEBOOK(target, source, *keys): 
    from json import load
    with open(source) as notebook:
        data = load(notebook)
        with open(target,'w') as module:
            for cell in data['cells']:
                lines = cell['source']
                if type(lines) == str:
                    lines = lines.split('\n')
                line0  = (lines or [''])[0].strip()
                haskey = lambda key: line0 == f'### {key} ###'
                if cell['cell_type'] == 'code' and any(map(haskey,keys)):
                    code = '\n'.join(lines)
                    module.write(code)
                    module.write('\n')
                     
MODULE_FROM_NOTEBOOK('./ttt_tactics.py',
                     './ttt-tactics.ipynb',
                     'HEAD','GAME','DATA','MODEL','SEARCH')

from ttt_tactics import *

!cat /proc/meminfo | grep Mem

ipynb = 'ttt-tactics-train'

# %load_ext tensorboard

# !rm -rf "./tensorboard/"


 composition.py   ttt-tactics		   ttt-tactics-test.ipynb
'!!!.ipynb'	  ttt-tactics.ipynb	   ttt-tactics-train-Copy2.ipynb
 __pycache__	  ttt-tactics-play.ipynb   ttt-tactics-train.ipynb
 ttt-3.ipynb	  ttt_tactics.py
INFO:tensorflow:Using local port 18538
INFO:tensorflow:Using local port 22416
INFO:tensorflow:Using local port 17762
INFO:tensorflow:Using local port 17034
INFO:tensorflow:Using local port 24085
INFO:tensorflow:Using local port 15886
INFO:tensorflow:Using local port 17884
INFO:tensorflow:Using local port 16692
INFO:tensorflow:Using local port 23486
INFO:tensorflow:Using local port 24338
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
cuda: False
tensorflow: 2.4.0
python: 3.8.5 (default, Jul 28 2020, 12:59:40) 
[GCC 9.3.0]
module:  ttt_tactics
DIR = ./ttt-tactics
<function savesamples at 0x7fdcbd6b5040>
<function loadsamples at 0x7fdcbd6b5280>
<class 'ttt_tactics.AlphaZeroModel'>
<function AlphaZeroPolicy at 0x7fdcbd6b5790>
MemTotal:       16393932 kB
MemF

In [2]:
def ConvLayer(filters     = None, 
              kernel_size = None, 
              strides     = None, 
              padding     = None,
              activation  = None):
    def proc(x, add=None):
        kwargs = dict(kernel_size = kernel_size,
                      strides     = strides,
                      padding     = padding,
                      data_format = 'channels_last',
                      use_bias    = False)
        if filters is None: 
            y = DepthwiseConv2D(depth_multiplier=1, **kwargs)(x)
        else: 
            y = Conv2D(filters=filters, **kwargs)(x)
        y = BatchNormalization()(y)
        if add is not None:
            y = Add()([y, add])  
        z = Activation(activation)(y)
        return z
    return proc

def InputLayer():
    def proc(x):
        z = ConvLayer(filters     = None, 
                      kernel_size = 3, 
                      strides     = 3, 
                      padding     = 'same',
                      activation  = 'relu')(x)
        return z
    return proc

def ValueHead():
    def proc(x):
        y = ConvLayer(filters     = 1, 
                      kernel_size = 1, 
                      strides     = 1, 
                      padding     = 'valid',
                      activation  = 'relu')(x)
        y = Flatten()(y)
        y = Dense(64, activation='relu')(y) 
        z = Dense(1, activation='tanh', name='value')(y) 
        return z
    return proc

def PolicyHead():
    def proc(x):
        y = ConvLayer(filters     = 2, 
                      kernel_size = 1, 
                      strides     = 1, 
                      padding     = 'valid',
                      activation  = 'relu')(x)
        y = Flatten()(y)
        z = Dense(num_a, activation='softmax', name='policy')(y)
        return z
    return proc
    
def LearningRateMetric(model):
    def metric(y_true, y_pred):
        return model.optimizer.lr
    metric.__name__ = 'lr'
    return metric    

from tensorflow.keras.layers import UpSampling2D, Concatenate 

class MyModel(Model):
    def __init__(self, residuals=5, path=None):
        x = Input((9,9,6,), name='state')
        y = InputLayer()(x)
        q = ValueHead()(y)
        y = UpSampling2D(size=3, data_format='channels_last', interpolation='nearest')(y)
        y = Concatenate()([x,y])
        p = PolicyHead()(y)
        super(MyModel, self).__init__(inputs=x, outputs=[q,p], name='alpha-zero-model')
        self.__name__ = self.name
        self.compile(
            loss         = { 'value': 'mse', 'policy': 'categorical_crossentropy' }, 
            loss_weights = { 'value': 1,     'policy': 1 },
            optimizer    = Adam(lr=0.001, clipnorm=1.0),
            metrics      = { 'value': [LearningRateMetric(self)] }) 
        if path is not None:
            self.load(path)  
    def clone():
        clone = MyModel(residuals=residuals)
        clone.set_weights(self.get_weights())
        return clone
    def softmax(self,pp,aa):
          return pp
    def __call__(self, s, training=False, policy=True):
        if training:
            q,ppp = super(MyModel, self).__call__(s, training=True)  
            return q,ppp
        aa    = actions(s)
        xx    = stack([onehot(s)])
        q,ppp = super(MyModel, self).__call__(xx, training=False)
        q,ppp = squeeze(q),squeeze(ppp)
        ii    = unravel(aa)
        pp    = gather_nd(ppp,ii)
        pp    = softmax(reshape(pp,[1,-1]))
        pp    = reshape(pp,[-1])
        if policy:            
            return aa,pp
        ppp   = zeros(num_a,pp.dtype)
        ppp   = update_nd(ppp,ii,pp)
        return q,ppp
    def __str__(self):
        return self.__name__
    def save(self, path):
        if isfile(path):
            MTIME(path)
            os.rename(path, path+'.bak')       
            MTIME(path+'.bak')
        self.save_weights(path, overwrite=True, save_format='h5')
        MTIME(path)
        self.__name__ = Path(path).stem
    def load(self, path):
        self.load_weights(path)
        MTIME(path)
        self.__name__ = Path(path).stem

print(MyModel)    

model = MyModel()
model.summary()


<class '__main__.MyModel'>
Model: "alpha-zero-model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
state (InputLayer)              [(None, 9, 9, 6)]    0                                            
__________________________________________________________________________________________________
depthwise_conv2d (DepthwiseConv (None, 3, 3, 6)      54          state[0][0]                      
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 3, 3, 6)      24          depthwise_conv2d[0][0]           
__________________________________________________________________________________________________
activation (Activation)         (None, 3, 3, 6)      0           batch_normalization[0][0]        
________________________________________________________

In [9]:
plot_model(model, show_shapes=False, show_layer_names=True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [3]:
model.save(f'{DIR}/model-00000000.h5')

./ttt-tactics/model-00000000.h5 Nov 06 2020 10:00:00
./ttt-tactics/model-00000000.h5.bak Nov 06 2020 10:00:00
./ttt-tactics/model-00000000.h5 Nov 06 2020 10:02:00


In [ ]:
minloss = 3.0
model   = MyModel()
models  = sortedfiles(f'{DIR}/model-*.h5')
model.load(models[-1])

left = 1000
while left > 0:
    left   -= 1
    path    = f'{DIR}/data-*.tfrec'
    files   = sortedfiles(path)
    dataset = loadsamples(files, buffer_size=200*1024, batch_size=1024, seed=None)
    print('path =',path)
    print('files:',len(files))

    schedule = { 0:0.0001 }
    def onschedule(epoch, lr): 
        if epoch in schedule: return schedule[epoch]  
        else:                 return lr
    scheduler = LearningRateScheduler(onschedule)
    
    @throttle(60*60)
    def onbatch(batch, logs):
        NOTIFY(f'{str(model)}, batch={batch}, logs={logs}', title=ipynb, priority=-1)
    def onepoch(epoch, logs):
        NOTIFY(f'{str(model)}, epoch={epoch}, logs={logs}', title=ipynb, priority=0)
    def ontrain(logs):
        ok = 'OK' if logs['loss'] < minloss else 'NO'
        NOTIFY(f'{str(model)}, train={ok}, logs={logs}', title=ipynb, priority=1)
    notifier = LambdaCallback(
        on_batch_end = onbatch,
        on_epoch_end = onepoch,
        on_train_end = ontrain)
    
    history = model.fit(
        x                   = dataset,
        epochs              = 10,
        verbose             = 1,
        callbacks           = [scheduler,notifier],
        workers             = 10,
        use_multiprocessing = False)
    history = history.history

    loss = history['loss'][-1]
    if loss < minloss:
        minloss = loss
        model.save(nextfile(models[-1]))

    plt.figure()
    plt.plot(history['loss'],        label='total loss')
    plt.plot(history['policy_loss'], label='policy loss')
    plt.plot(history['value_loss'],  label='value loss')
    plt.title('training history')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc="upper left")
    plt.savefig(f'{DIR}/{str(model)}.png')
    plt.show()


./ttt-tactics/model-00000000.h5 Nov 06 2020 10:02:00
Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.
Instructions for updating:
Do not call `graph_parents`.
path = ./ttt-tactics/data-*.tfrec
files: 100
Epoch 1/10
    944/Unknown - 76s 72ms/step - loss: 4.9572 - value_loss: 0.7883 - policy_loss: 4.1689 - value_lr: 1.0000e-04

In [ ]:
|